# Installs and imports

In [1]:
%pip install ijson
%pip install arabert


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: ijson in c:\users\kewi\appdata\local\programs\python\python310\lib\site-packages (3.2.0.post0)



You should consider upgrading via the 'c:\Users\kewi\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\kewi\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
import os
import json
import ijson
import pandas as pd
import os
import json
from transformers import AutoTokenizer
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import nltk
import seaborn as sns ,sklearn,gensim
from nltk.corpus import stopwords
from collections import  Counter
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kewi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Read the data from https://github.com/adelmeleka/AQAD

In [3]:
with open('AQAD-master/AQQAD 1.0/FINAL_AAQAD-v1.0.json','rb') as f:
  data = json.load(f)

for allData in data['data']:
    print(allData['title']+"\n")
    print(allData['paragraphs'])
    break

بيونسيه

[{'qas': [{'question': 'متى بدأت بيونسي تصبح شعبية؟', 'id': 1, 'answers': [{'text': 'في أواخر التسعينات', 'answer_start': 220}], 'is_impossible': False}, {'question': 'ما هي المجالات التي تنافس عليها بيونسيه عندما كانت تكبر؟', 'id': 2, 'answers': [{'text': 'غناء ورقص', 'answer_start': 173}], 'is_impossible': False}, {'question': 'متى غادرت بيونسي طفل القدر وتصبح مغنية منفردة؟', 'id': 3, 'answers': [{'text': '(2003)،', 'answer_start': 515}], 'is_impossible': False}, {'question': 'في أي مدينة و ولاية نشأت بيونسيه؟', 'id': 4, 'answers': [{'text': 'هيوستن بولاية تكساس،', 'answer_start': 88}], 'is_impossible': False}, {'question': 'في أي عقد أصبحت بيونسي مشهورة؟', 'id': 5, 'answers': [{'text': 'أواخر التسعينات', 'answer_start': 223}], 'is_impossible': False}, {'question': 'في أي مجموعة R&B كانت هي المغنية الرئيسية؟', 'id': 6, 'answers': [{'text': 'دستنيز تشايلد', 'answer_start': 290}], 'is_impossible': False}, {'question': 'ما الألبوم الذي جعلها فنانة معروفة عالميا؟', 'id': 7, 'ans

# Preprocessing

In [27]:
def parse_data(path):
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    # initialize lists for contexts, questions, and answers
    contexts = []
    questions = []
    answers = []
    # iterate through all data in squad data
    counter = 0
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                if 'plausible_answers' in qa.keys():
                    access = 'plausible_answers'
                else:
                    access = 'answers'
                for answer in qa[access]:
                    counter+=1
                    # append data to lists
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)
    # return formatted data lists
    print(counter)
    return contexts, questions, answers

In [28]:
contexts, questions, answers = parse_data('AQAD-master/AQQAD 1.0/FINAL_AAQAD-v1.0.json')
#splitting the data into train and test
from sklearn.model_selection import train_test_split
train_contexts, val_contexts, train_questions, val_questions, train_answers, val_answers = train_test_split(contexts, questions, answers, test_size=0.2, random_state=42)

17896


In [29]:
print(len(train_questions))
display(train_questions[0:3])
print(len(train_answers))
display(train_answers[0:3])
print(len(train_contexts))
display(train_contexts[0:3])
print('-----------------------------------------')
print(len(val_questions))
display(val_questions[0:3])
print(len(val_answers))
display(val_answers[0:3])
print(len(val_contexts))
display(val_contexts[0:3])

14316


['ما هي أكبر القطاعات الاقتصادية في برمودا؟',
 'من الذي أرسل ليحل محل المواقف المطرودة؟',
 'كم سنة يمكن تخزين الذاكرة على المدى القصير؟']

14316


[{'text': 'التأمين وإعادة التأمين والسياحة،', 'answer_start': 25},
 {'text': 'الروس', 'answer_start': 192},
 {'text': 'من', 'answer_start': 24}]

14316


...

['ماذا يجب أن يحدث إذا ركزت فقط على إرضاء الذات؟',
 'ما هي الأعراض التي ستحدث إذا تسبب مرض السل في تمدد الأوعية الدموية في راسموسن؟',
 'ما ينمو على نقل الأنهار الجليدية في ألاسكا؟']

3580


[{'text': 'بيد أن القاعدة التي يتم خرقها ربما لا تكون', 'answer_start': 330},
 {'text': 'نزف', 'answer_start': 367},
 {'text': 'النباتات', 'answer_start': 73}]

3580


['إن فعل العصيان المدني هو خرق واع ومتعمد للقانون. كما أنه انتهاك لقاعدة قانونية وضعية. فإذا كان الانتهاك يقوم على القاعدة محل الخلاف مباشرة، فنحن بصدد الحديث عن عصيان مباشر. وهذا هو الحال، على سبيل المثال، بالنسبة لحملة العصيان المدني التي أطلقها مارتن لوثر كينغ والتي كانت تهدف إلى شغل السود للأماكن المخصصة بمقتضى للقانون للبيض. بيد أن القاعدة التي يتم خرقها ربما لا تكون هي تلك محل الخلاف، وحينئذ يكون الحديث عن عصيان مدني غير مباشر، وهو الحال- على سبيل المثال- بالنسبة للاعتصامات التي لا تهدف إلى الاحتجاج على قانون المرور.\nوعلى الرغم من عدم إمكانية إدراك وجود خرق مسبق (فالقاضي وحده هو الذي يحدد وجود خرق من عدمه)، فإن أي فعل يعد عصيانا مدنيا، عندما يخاطر القائمون عليه بعمل يكون، في نظر الرأي العام ورأي السلطات، خرقا عاما للقانون.\nوعند التعرض لهذه القضية، يجدر التذكير بالتجربة التي قام بها ستانلي ميلجرام. والتي تمثلت في قياس نسبة الأفراد القادرين على إطلاق مثل هذا العمل الخاص بالعصيان على الرغم من وجود ضغوط اجتماعية وإدارية.\n',
 'إذا تحولت عدوى السل إلى حالة نشطة، فإنها ستُصيب الرئتين

## Preprocessing the dataset using arabert in addition to this SQuAD preprocessing guide https://www.youtube.com/watch?v=ZIRmXkHp0-c&t=287s

In [30]:
def add_end_idx(answers, contexts):
    # loop through each answer-context pair
    for answer, context in zip(answers, contexts):
        # gold_text refers to the answer we are expecting to find in context
        gold_text = answer['text']
        # we already know the start index
        start_idx = answer['answer_start']
        # and ideally this would be the end index...
        end_idx = start_idx + len(gold_text)

        # ...however, sometimes squad answers are off by a character or two
        if context[start_idx:end_idx] == gold_text:
            # if the answer is not off :)
            answer['answer_end'] = end_idx
        else:
            for n in [1, 2]:
                if context[start_idx-n:end_idx-n] == gold_text:
                    # this means the answer is off by 'n' tokens
                    answer['answer_start'] = start_idx - n
                    answer['answer_end'] = end_idx - n
add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [31]:
#loop over answers, and contexts and make sure that they match
def check_answers_contexts_match_and_find_anomalies(answers, contexts, questions, show_anomalies=False):
    
    answers_without_answer_end_key =0
    non_matching_answers=0
    indices_of_answers_without_answer_end_key = []
    for answer, context, question in zip(answers, contexts, questions):
        
        #check if context has key answer_end and if not 
        if 'answer_end' not in answer.keys():
            answers_without_answer_end_key +=1
            if show_anomalies:
                print(question)
                print(answer)
                print(context[answer['answer_start']:answer['answer_start']+ len(answer['text'])+1])
            indices_of_answers_without_answer_end_key.append(answers.index(answer))

            continue 

        #check if answer matches context
        if context[answer['answer_start']:answer['answer_end']] != answer['text']:
            non_matching_answers +=1
    display('answers without answer_end key:', answers_without_answer_end_key,'non matching answers:', non_matching_answers)
    return indices_of_answers_without_answer_end_key
    
indices_to_drop1 = check_answers_contexts_match_and_find_anomalies(train_answers, train_contexts, train_questions)

#drop the answers, contexts and questions that do not match the context
train_answers = [i for j, i in enumerate(train_answers) if j not in indices_to_drop1]
train_contexts = [i for j, i in enumerate(train_contexts) if j not in indices_to_drop1]
train_questions = [i for j, i in enumerate(train_questions) if j not in indices_to_drop1]
indices_to_drop2 = check_answers_contexts_match_and_find_anomalies(val_answers, val_contexts, val_questions)
#drop the answers, contexts and questions that do not match the context
val_answers = [i for j, i in enumerate(val_answers) if j not in indices_to_drop2]
val_contexts = [i for j, i in enumerate(val_contexts) if j not in indices_to_drop2]
val_questions = [i for j, i in enumerate(val_questions) if j not in indices_to_drop2]
# print length of data after dropping
print(len(indices_to_drop1))
print(len(indices_to_drop2))
print(len(train_questions))
print(len(train_answers))
print(len(train_contexts))
print(len(val_questions))
print(len(val_answers))
print(len(val_contexts))


'answers without answer_end key:'

57

'non matching answers:'

0

'answers without answer_end key:'

7

'non matching answers:'

0

57
7
14271
14271
14271
3573
3573
3573


dropping the triplets which don't have answer_end in their answers

## Encoding

In [32]:
arabert_tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabert",
                                                    do_lower_case=False,
                                                    do_basic_tokenize=True,
                                                    remove_html_markup = False,
                                                    strip_tashkeel = True,
                                                    strip_tatweel = True,
                                                    keep_emojis = True,
                                                    insert_white_spaces = True,
                                                    replace_slash_with_dash = True,
                                                    apply_farasa_segmentation = True)

In [33]:
def show_encoding_of(sentence, tokenizer):
    encoding = tokenizer(sentence)
    
    print('length of encoding:', len(encoding['input_ids']))
    print(' number of words around splitted spaces :', len(sentence.split(' ')))
    list_of_tokens = []
    count_of_unk_tokens = 0
    for i in encoding['input_ids']:
        #print the decoding of it
        list_of_tokens.append(tokenizer.decode([i]))
        if(tokenizer.decode([i]) == '[UNK]'):
            count_of_unk_tokens +=1
    #print the list of tokens
    print('number of tokens that are not in the vocab:', count_of_unk_tokens)
    print(sentence)
    # loop over the list of tokens and print them
    # for token in list_of_tokens:
    #     display(token)
    print(list_of_tokens[1:-1])
    # print(encoding)
    
# test_text = "الألواح الخشبية تعالج بعد ذلك في أفران للتحميص حتى لا تتأثر مستقبلاً بتغييرات درجات الحرارة أو الرطوبة الموجودة في الجو"
# test_text = "يوجد في باريس العديد من الملاعب المخصصة لمختلف أنواع الرياضات. يعدّ ملعب فرنسا الذي يتسع لأكثر من 80 ألف متفرج أكبر ملاعب البلاد، وكان قد بني هذا الملعب الواقع في منطقة سان دينس لاستضافة كأس العالم لكرة القدم 1998 والذي حازت عليه فرنسا للمرة الأولى في تاريخها. يستخدم الملعب لممارسة كرة القدم، والرجبي، وألعاب القوى. يستضيف الملعب مباريات منتخب فرنسا الوطني للرجبي سنوياً في بطولة الأمم الستة. وكذلك يستضيف مباريات منتخب فرنسا لكرة القدم الودية وتصفيات البطولات الكبرى. بالإضافة إلى نادي باريس سان جيرمان، تملك المدينة نوادي كرة قدم أخرى مثل: نادي باريس، والنجم الأحمر ونادي فرنسا"
for i in range(5,10):
    test_text = train_contexts[i]
    show_encoding_of(test_text, arabert_tokenizer)


length of encoding: 188
 number of words around splitted spaces : 98
number of tokens that are not in the vocab: 3
توضع الوحدات الصخرية بداية إما عن طريق الترسيب على سطح ما أو عن طريق استيلاءها على سطح صخر ما وذلك بوجودها في علوية ذلك الصخر. يمكن أن يحدث الترسيب في حال استقرار الرواسب على سطح الأرض وتشكل طبقات عن طريق تصلبها وتحولها من رسوبيات إلى صخور رسوبية صلبة أو في حال مادة بركانية كالرماد البركاني أو الحمم البركانية التي تتدفق في باطن الأرض، تعتبر الرواسب النارية مثل: الباثوليثات أجساماً نارية عميقة ويكون شكلها غير منتظم، وكذلك تعتبر اللاكوليثات أجساماً نارية ضحلة ويكون شكلها منتظماً، أما القواطع (دايكات) والطبقات الافقية (سيلات) تترسب وتتجمع في علوية صخرها وتتبلور كما تطفلت.

['توضع', 'ال', '##وحد', '##ات', 'ال', '##صخر', '##ي', '##ة', 'بداي', '##ة', 'إما', 'عن', 'طريق', 'ال', '##ترس', '##يب', 'على', 'سطح', 'ما', 'أو', 'عن', 'طريق', 'استيلاء', '##ها', 'على', 'سطح', 'صخر', 'ما', 'و', '##ذلك', 'بوج', '##ود', '##ها', 'في', 'علوي', '##ة', 'ذلك', 'ال', '##صخر', '.', 'يمكن', 'أن', 'يح

In [34]:
tokenizer = arabert_tokenizer
train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

In [35]:
def add_token_positions(encodings, answers):
    # initialize lists to contain the token indices of answer start/end
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        # append start/end token position using char_to_token method
        
        if('answer_end' not in answers[i]):
            continue
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']))
        

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        # end position cannot be found, char_to_token found space, so shift one token forward
        go_back = 1
        print(end_positions[-1])
        while end_positions[-1] is None:
            end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end']-go_back)
            go_back +=1
    # update our encodings object with the new token-based start/end positions
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

# apply function to our data
add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)

None
None
None
None
None
None
None
None
139
None
None
17
None
None
None
None
None
None
None
None
None
None
None
None
None
413
None
None
None
None
None
None
None
None
None
None
54
126
None
89
None
None
None
None
None
None
83
None
None
None
195
None
None
None
None
None
None
None
None
None
None
None
None
None
None
136
None
None
None
None
210
None
None
None
None
None
None
11
None
None
None
None
None
None
None
None
None
None
None
111
None
None
None
None
None
None
None
None
36
None
None
None
None
27
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
20
None
63
None
None
None
83
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
225
43
None
None
None
None
None
None
None
None
None
None
58
None
None
None
None
None
185
None
None
None
None
None
None
135
None
69
None
31
None
None
None
None
None
None
None
None
None
17
227
None
144
None
None
None
No

In [36]:
train_encodings['start_positions'][0:20]

[6,
 64,
 10,
 116,
 15,
 170,
 26,
 75,
 136,
 52,
 104,
 13,
 73,
 174,
 60,
 111,
 128,
 20,
 105,
 41]

# Importing the model

In [37]:
import torch

class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)
from transformers import DistilBertForQuestionAnswering
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")

Downloading:  18%|█▊        | 45.0M/256M [05:09<1:07:19, 54.7kB/s] 

# Finetuning

In [ ]:
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm

# setup GPU/CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# move model over to detected device
model.to(device)
# activate training mode of model
model.train()
# initialize adam optimizer with weight decay (reduces chance of overfitting)
optim = AdamW(model.parameters(), lr=5e-5)

# initialize data loader for training data
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

for epoch in range(3):
    # set model to train mode
    model.train()
    # setup loop (we use tqdm for the progress bar)
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all the tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        # train model on batch and return outputs (incl. loss)
        outputs = model(input_ids, attention_mask=attention_mask,
                        start_positions=start_positions,
                        end_positions=end_positions)
        # extract loss
        loss = outputs[0]
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

# Saving the model

In [ ]:
model_path = 'models/distilbert-custom'
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

# Evaluation

In [ ]:
# switch model out of training mode
model.eval()

#val_sampler = SequentialSampler(val_dataset)
val_loader = DataLoader(val_dataset, batch_size=16)

acc = []

# initialize loop for progress bar
loop = tqdm(val_loader)
# loop through batches
for batch in loop:
    # we don't need to calculate gradients as we're not training
    with torch.no_grad():
        # pull batched items from loader
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_true = batch['start_positions'].to(device)
        end_true = batch['end_positions'].to(device)
        # make predictions
        outputs = model(input_ids, attention_mask=attention_mask)
        # pull preds out
        start_pred = torch.argmax(outputs['start_logits'], dim=1)
        end_pred = torch.argmax(outputs['end_logits'], dim=1)
        # calculate accuracy for both and append to accuracy list
        acc.append(((start_pred == start_true).sum()/len(start_pred)).item())
        acc.append(((end_pred == end_true).sum()/len(end_pred)).item())
# calculate average accuracy in total
acc = sum(acc)/len(acc)

In [ ]:
print("T/F\tstart\tend\n")
for i in range(len(start_true)):
    print(f"true\t{start_true[i]}\t{end_true[i]}\n"
          f"pred\t{start_pred[i]}\t{end_pred[i]}\n")